In [97]:
from selenium import webdriver
import datetime, re, requests, io, time, random, string
from bs4 import BeautifulSoup
from credentials import email, password
from pymongo import MongoClient
import datetime
client = MongoClient('localhost', 27017)

In [47]:
urls = {
    'sign_in': 'https://wallmine.com/users/sign-in',
    'homepage': 'https://wallmine.com/',
    'view_stock': lambda exchange, symbol: f'https://wallmine.com/{exchange}/{symbol}'
}
urls['view_stock']('nasdaq','tsla')

'https://wallmine.com/nasdaq/tsla'

In [48]:
driver = webdriver.Chrome('/Users/nikkisalehi/downloads/chromedriver')

In [49]:
def login():
    '''Method used to login into Wallmine account'''
    driver.get('https://wallmine.com/users/sign-in')
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[1]/div[2]/a').click()
    time.sleep(0.5)
    driver.find_element_by_xpath('//*[@id="user_email"]').send_keys(email)
    driver.find_element_by_xpath('//*[@id="user_password"]').send_keys(password)
    time.sleep(0.1)
    driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[2]/div[1]/button').click()
    time.sleep(3)
    if driver.find_element_by_xpath('/html/body/main/section/div[2]/div/div[1]/h1/span'):
        print('Login was successful')
        

In [50]:
login()

Login was successful


In [73]:
  def retrieve_data():
    '''Method use to retrieve stock data from Wallmine'''
    # we need to be on the site after login in order for this method to work
    driver.find_element_by_xpath('//*[@id="homepage-heatmap"]/a/div[2]').click()
    time.sleep(5)
    text = driver.find_element_by_xpath('/html/body/main/section/div[4]/div/div/div[1]/div/div[1]/h1').text
    time.sleep(5)
    if text == "Free Stock Screener":
        print('On the right page')
     
    driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[1]/div/ul/li[1]/a').click()
    time.sleep(5) 
    table_head = driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[2]/table/thead').text
    table_body = driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[2]/table/tbody').text
    time.sleep(2) 
    raw_data = [table_head, table_body]
    
    return raw_data

In [74]:
got_data = retrieve_data()
got_data

On the right page


['▴ ▴\nSymbol\n▴ ▴\nCompany\n▴ ▴\nExchange\n▴ ▴\nIndustry\n▴ ▴\nMarket cap\n▴ ▴\nEBITDA\n▴ ▴\nP/E\n▴ ▴\nEV/EBITDA\n▴ ▴\nDebt/Equity\n▴ ▴\nAverage volume\n▴ ▴\nInstitutional ownership\n▴ ▴\nEarnings date\n▴ ▴\nPrice\n▴ ▴\nPerformance today',
 'AAPL Apple Inc NASDAQ Consumer Electronics $1.97T $85.159B 32.21 24.07 3.96 114.562M 59.66% 27 Jan 2021 $121.21 +0.51%\nMSFT Microsoft Corporation NASDAQ Internet Services & Infrastructure $1.76T $60.155B 34.25 27.69 1.55 30.361M 71.84% 26 Jan 2021 $236.48 +1.78%\nAMZN Amazon.com Inc. NASDAQ Specialty Retail $1.52T $48.091B 71.44 31.10 2.44 4.102M 58.66% 2 Feb 2021 $3,052.03 +0.19%\nGOOGL Alphabet Inc NASDAQ Internet Services & Infrastructure $1.38T $55.322B 34.36 22.72 0.44 N/A 80.25% 2 Feb 2021 $2,024.73 -0.38%\nGOOG Alphabet Inc NASDAQ Internet Services & Infrastructure $1.38T N/A 41.23 N/A 0.44 1.613M 68.10% 2 Feb 2021 $2,035.55 -0.43%\nFB Facebook Inc NASDAQ Internet Services & Infrastructure $826.14B $39.533B 27.27 19.38 0.24 18.669M 79.77% 

In [75]:
got_data[0]

'▴ ▴\nSymbol\n▴ ▴\nCompany\n▴ ▴\nExchange\n▴ ▴\nIndustry\n▴ ▴\nMarket cap\n▴ ▴\nEBITDA\n▴ ▴\nP/E\n▴ ▴\nEV/EBITDA\n▴ ▴\nDebt/Equity\n▴ ▴\nAverage volume\n▴ ▴\nInstitutional ownership\n▴ ▴\nEarnings date\n▴ ▴\nPrice\n▴ ▴\nPerformance today'

In [77]:
clean_data_head = got_data[0].split('\n')
clean_data_head

['▴ ▴',
 'Symbol',
 '▴ ▴',
 'Company',
 '▴ ▴',
 'Exchange',
 '▴ ▴',
 'Industry',
 '▴ ▴',
 'Market cap',
 '▴ ▴',
 'EBITDA',
 '▴ ▴',
 'P/E',
 '▴ ▴',
 'EV/EBITDA',
 '▴ ▴',
 'Debt/Equity',
 '▴ ▴',
 'Average volume',
 '▴ ▴',
 'Institutional ownership',
 '▴ ▴',
 'Earnings date',
 '▴ ▴',
 'Price',
 '▴ ▴',
 'Performance today']

In [78]:
def remove_characters(x):
    if x == '▴ ▴':
        return False
    else:
        return True

In [81]:
categories = list(filter(remove_characters, clean_data_head))
categories

['Symbol',
 'Company',
 'Exchange',
 'Industry',
 'Market cap',
 'EBITDA',
 'P/E',
 'EV/EBITDA',
 'Debt/Equity',
 'Average volume',
 'Institutional ownership',
 'Earnings date',
 'Price',
 'Performance today']

In [84]:
clean_data_body = got_data[1].split('\n')
clean_data_body[0]

'AAPL Apple Inc NASDAQ Consumer Electronics $1.97T $85.159B 32.21 24.07 3.96 114.562M 59.66% 27 Jan 2021 $121.21 +0.51%'

In [85]:
split_company = clean_data_body[0].split('$')
split_company

['AAPL Apple Inc NASDAQ Consumer Electronics ',
 '1.97T ',
 '85.159B 32.21 24.07 3.96 114.562M 59.66% 27 Jan 2021 ',
 '121.21 +0.51%']

In [87]:
# TODO Make an object with the following:
# Symbol
# Company Name
# Exchange
# Price
# Store all of these inside of an object
{
    'symbol' 'AAPL',
    'company_name': 'Apple Inc',
    'exchange': 'NASDAQ',
    'price': '121.21'
}

SyntaxError: invalid syntax (<ipython-input-87-df126c6a80ca>, line 9)

In [98]:
def parse_company(split_company):
    result = {}
    split_row_one = split_company[0].split(' ')
    split_row_two = split_company[1].split(' ')
    split_row_three = split_company[2].split(' ')
    split_row_four = split_company[3].split(' ')
    stop = ['NASDAQ', 'NYSE', 'NYSEMKT']
    symbol = split_row_one[0]
    price = split_row_four[0]
    for n in stop:
        if n in split_company[0]:
            exchange = n
    a = split_row_one.index(exchange)
    company_name_arr = split_row_one[1:a]
    company_name = ' '.join(str(e) for e in company_name_arr)
    result['company_name'] = company_name
    result['exchange'] = exchange
    result['symbol'] = symbol
    result['price'] = price
    result['date'] = datetime.datetime.now()
    return result # object/dictionary
    


In [100]:
all_company_data = []
for stock in clean_data_body:
    if 'N/A' in stock:
        continue
    elif '¥' in stock:
        continue
    elif '€' in stock:
        continue
    split_company = stock.split('$')
    x = parse_company(split_company)
    all_company_data.append(x)

In [101]:
final_list = [categories, all_company_data]
final_list

[['Symbol',
  'Company',
  'Exchange',
  'Industry',
  'Market cap',
  'EBITDA',
  'P/E',
  'EV/EBITDA',
  'Debt/Equity',
  'Average volume',
  'Institutional ownership',
  'Earnings date',
  'Price',
  'Performance today'],
 [{'company_name': 'Apple Inc',
   'exchange': 'NASDAQ',
   'symbol': 'AAPL',
   'price': '121.21',
   'date': datetime.datetime(2021, 3, 26, 16, 55, 10, 630274)},
  {'company_name': 'Microsoft Corporation',
   'exchange': 'NASDAQ',
   'symbol': 'MSFT',
   'price': '236.48',
   'date': datetime.datetime(2021, 3, 26, 16, 55, 10, 630288)},
  {'company_name': 'Amazon.com Inc.',
   'exchange': 'NASDAQ',
   'symbol': 'AMZN',
   'price': '3,052.03',
   'date': datetime.datetime(2021, 3, 26, 16, 55, 10, 630294)},
  {'company_name': 'Facebook Inc',
   'exchange': 'NASDAQ',
   'symbol': 'FB',
   'price': '282.96',
   'date': datetime.datetime(2021, 3, 26, 16, 55, 10, 630300)},
  {'company_name': 'Berkshire Hathaway Inc.',
   'exchange': 'NYSE',
   'symbol': 'BRK-B',
   'pri